In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


# Importing Libraries 

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

# Config Hyperparameter

In [5]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

dataset_name = "utkarshpophli/mathematics_dataset"

new_model = "llama-2-7b-math-finetune"

# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

openai/gsm8k

In [10]:
# Load dataset
dataset = load_dataset(dataset_name, split="train[:20%]")

In [11]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Checking GPU compatibility
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Loading base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Loading LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Loading LoRA config
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/17432 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,3.172000
50,5.808300
75,2.712100
100,3.853700
125,1.868000
150,3.240900
175,1.649100
200,3.186500
225,1.604900
250,2.925700


TrainOutput(global_step=4358, training_loss=1.6439657434513395, metrics={'train_runtime': 6094.0377, 'train_samples_per_second': 2.861, 'train_steps_per_second': 0.715, 'total_flos': 1.217577738338304e+16, 'train_loss': 1.6439657434513395, 'epoch': 1.0})

In [12]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [13]:
!zip -r /kaggle/working/llama_math_results_tensorboard.zip /kaggle/working/results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/results/ (stored 0%)
  adding: kaggle/working/results/runs/ (stored 0%)
  adding: kaggle/working/results/runs/Jul07_08-34-38_72ee8fb3ce9d/ (stored 0%)
  adding: kaggle/working/results/runs/Jul07_08-34-38_72ee8fb3ce9d/events.out.tfevents.1720341295.72ee8fb3ce9d.34.1 (deflated 60%)
  adding: kaggle/working/results/runs/Jul07_08-42-21_72ee8fb3ce9d/ (stored 0%)
  adding: kaggle/working/results/runs/Jul07_08-42-21_72ee8fb3ce9d/events.out.tfevents.1720341755.72ee8fb3ce9d.34.2 (deflated 68%)
  adding: kaggle/working/results/runs/Jul07_08-30-11_72ee8fb3ce9d/ (stored 0%)
  adding: kaggle/working/results/runs/Jul07_08-30-11_72ee8fb3ce9d/events.out.tfeve

In [ ]:
import shutil
shutil.copy('/content/llama_math_results_tensorboard.zip', '/content/drive/MyDrive/llama_math_results_tensorboard.zip')

In [15]:
!pip install tensorboard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
%load_ext tensorboard
%tensorboard --logdir results/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 203), started 0:01:57 ago. (Use '!kill 203' to kill it.)

In [20]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Find the largest number from all natural numbers less than or equal to 100, which become common multiples of 7 and 8 if 2 is subtracted."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Find the largest number from all natural numbers less than or equal to 100, which become common multiples of 7 and 8 if 2 is subtracted. [/INST] Solution: Let $n$ be the largest number less than or equal to 100 that becomes a common multiple of 7 and 8 when $2$ is subtracted. We can write $n = 7m + 8$ for some integer $m$. Subtracting $2$ from both sides gives $n = 7m + 8 - 2 = 7m + 6$. Since $n$ is less than or equal to 100, $m$ must be less than or equal to 100/7 = 14. Therefore, $n = 7m + 6 = 7(14) + 6 = \boxed{98}$.</s>


In [21]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

20772

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model.push_to_hub("utkarshpophli/Llama-2-7b-math-finetune", check_pr=True)

tokenizer.push_to_hub("utkarshpophli/Llama-2-7b-math-finetune",check_pr=True)